In [1]:
import keras

In [2]:
from keras import layers

In [3]:
import numpy as np

In [4]:
import matplotlib.pyplot as plt

In [5]:
latent_dim = 32
height = 32
width = 32
channels = 3

In [6]:
generator_input = keras.Input(shape=(latent_dim,))

In [7]:
x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)
generator = keras.models.Model(generator_input, x)

2022-02-08 14:04:22.334980: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-08 14:04:22.399202: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-08 14:04:22.399383: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-08 14:04:22.399897: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the app

In [8]:
generator.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32)]              0         
                                                                 
 dense (Dense)               (None, 32768)             1081344   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 32768)             0         
                                                                 
 reshape (Reshape)           (None, 16, 16, 128)       0         
                                                                 
 conv2d (Conv2D)             (None, 16, 16, 256)       819456    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 16, 16, 256)       0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 32, 32, 256)      104883

In [9]:
import tensorflow

In [10]:
discriminator_input = layers.Input(shape=(height, width, channels))
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(1, activation='sigmoid')(x)
discriminator = tensorflow.keras.models.Model(discriminator_input, x)
discriminator_optimizer = tensorflow.keras.optimizers.RMSprop(learning_rate=0.0008,clipvalue=1.0,decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer,loss='binary_crossentropy')


In [11]:
discriminator.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 30, 30, 128)       3584      
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 30, 30, 128)       0         
                                                                 
 conv2d_5 (Conv2D)           (None, 14, 14, 128)       262272    
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 14, 14, 128)       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 6, 6, 128)         262272    
                                                                 
 leaky_re_lu_7 (LeakyReLU)   (None, 6, 6, 128)         0   

In [12]:
discriminator.trainable = False
gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)
gan_optimizer = tensorflow.keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

/usr/lib/python3.10/site-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [13]:
gan.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 32)]              0         
                                                                 
 model (Functional)          (None, 32, 32, 3)         6264579   
                                                                 
 model_1 (Functional)        (None, 1)                 790913    
                                                                 
Total params: 7,055,492
Trainable params: 6,264,579
Non-trainable params: 790,913
_________________________________________________________________


In [14]:
import os

In [15]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing import image

In [16]:
(train_x , train_y) , (test_x , test_y) = cifar10.load_data()

In [17]:
train_x = train_x[train_y.flatten() == 6]

In [18]:
train_x.shape

(5000, 32, 32, 3)

In [19]:
train_x = train_x.reshape((-1,height,width,channels)).astype('float32') / 255

In [20]:
iterations = 10000

In [21]:
batch_size=20

In [26]:
save_dir = 'frogs'

In [27]:
start = 0
for step in range(iterations):
    random_latent_vectors = np.random.normal(size=(batch_size,latent_dim))
    generated_images = generator.predict(random_latent_vectors)
    stop = start + batch_size
    real_images = train_x[start:stop]
    
    combined_images = np.concatenate([generated_images,real_images])
    labels = np.concatenate([np.ones((batch_size,1)),
                            np.zeros((batch_size,1))])
    labels += 0.05 * discriminator.train_on_batch(combined_images , labels)
    d_loss = discriminator.train_on_batch(combined_images , labels)
    
    random_latent_vectors = np.random.normal(size=(batch_size,latent_dim))
    misleading_targets = np.zeros((batch_size,1))
    a_loss = gan.train_on_batch(random_latent_vectors,misleading_targets)
    
    start += batch_size
    if start > len(train_x) - batch_size:
        start = 0
    if step % 100 == 0:
        gan.save_weights('gan.h5')
        
        print('discriminator loss', d_loss)
        print('adverserial loss' , a_loss)
        
        img = image.array_to_img(generated_images[0] * 255 , scale=False)
        img.save(os.path.join(save_dir,'generated_frog' + str(step) + '.png'))
        
        img = image.array_to_img(real_images[0] * 255 , scale=False)2
        img.save(os.path.join(save_dir , 'real_frog' + str(step) + '.png'))
        

discriminator loss 0.3317388594150543
adverserial loss 1.6023927927017212
discriminator loss 0.39606648683547974
adverserial loss 0.9372115135192871
discriminator loss 0.5347632765769958
adverserial loss 1.2676079273223877
discriminator loss 0.4621354937553406
adverserial loss 1.744518518447876
discriminator loss 0.45870381593704224
adverserial loss 1.0795338153839111
discriminator loss 0.5174083113670349
adverserial loss 0.9963424801826477
discriminator loss 0.49775823950767517
adverserial loss 0.7556710839271545
discriminator loss 0.6083162426948547
adverserial loss 0.8025890588760376
discriminator loss 0.48227959871292114
adverserial loss 0.5353943705558777
discriminator loss 0.34673720598220825
adverserial loss 0.9571229815483093
discriminator loss 0.46261292695999146
adverserial loss 1.4432169198989868
discriminator loss 0.4333380162715912
adverserial loss 1.1500861644744873
discriminator loss 0.3510391414165497
adverserial loss 1.3802120685577393
discriminator loss 0.548657894134